#    Notebook para procesar archivos de voz v0010201

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

!pip install pytube

!apt -qq install -y sox
!apt -qq install -y sox libsox-fmt-mp3
!pip install sox

!git clone https://github.com/rabitt/pysox.git
!cd /content/pysox
!python /content/pysox/setup.py install

!pip install git+https://github.com/rabitt/pysox.git

!pip3 install ffmpeg
!apt -qq install -y ffmpeg
!pip3 install pydub

!pip3 install xlrd
!pip3 install --upgrade xlrd

!pip3 install pandas
!pip3 install --upgrade pandas



Mounted at /content/drive/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.9 MB/s 
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 20 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack ...

In [3]:
!pip install xlrd
!pip3 install xlrd
!pip install --upgrade --force-reinstall xlrd
!pip3 install pandas
!pip3 install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import shutil
import numpy as np
import tensorflow as tf
import pandas

from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
from google.colab import drive
drive.mount('/content/drive/')

# Variables de sesión
#DATASET_ROOT = "/home/jmmiguez/proyectoAudio/audio/"
DATASET_ROOT = '/content/drive/MyDrive/proyectoAudio/audio'

# Las carpetas en las cuales voy a poner los ejemplos de audio y los ejemplos de ruidos
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"
DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

print ("DATASET_ROOT {}".format(DATASET_ROOT))
print ("DATASET_AUDIO_PATH {}".format(DATASET_AUDIO_PATH))
print ("DATASET_NOISE_PATH {}".format(DATASET_NOISE_PATH))

# Porcentaje de muestras que voy a usar para validación
VALID_SPLIT = 0.2

# Semilla que voy a usar para mezclar los datos con el ruido
SHUFFLE_SEED = 43

# La tasa de muestreo a usar es única para todas las muestras de audio.
# Se vuelve a muestrear todo el ruido a esta frecuencia de muestreo.
# Este también será el tamaño de salida de las muestras de las señales de audio.
# (ya que todas las muestras son de 1 segundo de duración)
SAMPLING_RATE = 16000

# El factor a multiplicar por el ruido, es acorde a:
#   muestra_de_ruido = muestreo + ruido * prop * escala
#      donde prop = amplitud_de_muestreo / amplitud_de_ruido
SCALE = 0.5

BATCH_SIZE = 128
#EPOCHS = 100
EPOCHS = 50

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
DATASET_ROOT /content/drive/MyDrive/proyectoAudio/audio
DATASET_AUDIO_PATH /content/drive/MyDrive/proyectoAudio/audio/audio
DATASET_NOISE_PATH /content/drive/MyDrive/proyectoAudio/audio/noise


In [5]:
# Si la carpeta 'audio' no existe, la creo; caso contrario, no hago nada.
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)

# si la carpeta 'noise' no existe, la creo; caso contrario, no hago nada.
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        
        print ("folder = {}".format(folder))
        
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # Si la carpeta es 'audio' o 'noise', no hago nada
            continue
        elif folder in ["other", "_background_noise_"]:
            # Si la sub-carpeta es una de las que contienen muestras de ruido,
            # moverla a la carpeta 'noise'
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # De otra forma, debe ser una carpeta de un hablante, asique hay que moverla a la carpeta de 'audio'
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

folder = audio
folder = noise
folder = Downloads


In [6]:
# Obtengo la lista de todos los archivos de ruido
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir

    print("subdir_path= {}".format(subdir_path))

    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print(
    "En el directorio {}, se encontró {} archivos, pertenecientes a {} directorios".format(
        DATASET_NOISE_PATH, len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

print(noise_paths)

subdir_path= /content/drive/MyDrive/proyectoAudio/audio/noise/audio
subdir_path= /content/drive/MyDrive/proyectoAudio/audio/noise/noise
En el directorio /content/drive/MyDrive/proyectoAudio/audio/noise, se encontró 3 archivos, pertenecientes a 2 directorios
['/content/drive/MyDrive/proyectoAudio/audio/noise/audio/bicicleta.wav', '/content/drive/MyDrive/proyectoAudio/audio/noise/noise/aplausos.wav', '/content/drive/MyDrive/proyectoAudio/audio/noise/noise/ruido.wav']


In [7]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

# Dividir el ruido en fragmentos de 16000 Hz cada uno
def load_noise_sample(path):
    
    print(path)
    
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == SAMPLING_RATE:
        # Número de cortes a 16000 cada uno, que se pueden generar a partir de la muestra de ruido
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("La tasa de muestrea para {} es incorrecta. Se ignora".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print( "format(len(noise_paths)=           {}".format(len(noise_paths)) )
print( "noises.shape[0]=                   {}".format(noises.shape[0]) )
print( "noises.shape[1]=                   {}".format(noises.shape[1]) )
print( "noises.shape[1] // SAMPLING_RATE = {}".format(noises.shape[1] // SAMPLING_RATE) )

print(
    "{} archivos de ruido fueron divididos en {} muestras de ruido, donde cada una tiene {} seg. de tiempo.".format(
        len(noise_paths),noises.shape[0],noises.shape[1] // SAMPLING_RATE)
)

/content/drive/MyDrive/proyectoAudio/audio/noise/audio/bicicleta.wav
/content/drive/MyDrive/proyectoAudio/audio/noise/noise/aplausos.wav
/content/drive/MyDrive/proyectoAudio/audio/noise/noise/ruido.wav
format(len(noise_paths)=           3
noises.shape[0]=                   137
noises.shape[1]=                   16000
noises.shape[1] // SAMPLING_RATE = 1
3 archivos de ruido fueron divididos en 137 muestras de ruido, donde cada una tiene 1 seg. de tiempo.


In [8]:

def paths_and_labels_to_dataset(audio_paths, labels):
    """Construir un dataset de audios y etiquetas."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Leer y decodificar un archivo de audio."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Crear un tensor aleatorio del mismo tamaño que el audio que va 
        # desde 0 hasta la cantidad de muestras de flujo de ruido que tenemos.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Obtener la proporción de amplitud entre el audio y el ruido.
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Agregar el ruido reescalado al audio
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    # Dado que tf.signal.fft aplica FFT en la dimensión más interna, 
    # debemos comprimir las dimensiones y luego expandirlas nuevamente 
    # después de FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Devuelve el valor absoluto de la primera mitad de la FFT 
    # que representa las frecuencias positivas
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])


# Obtener la lista de rutas de archivos de audio junto con sus etiquetas correspondientes

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Nombres de nuestras Clases: {}".format(class_names,))

audio_paths = []
labels = []

#Reemplazo la recorrida original de archivos de audio, por los nombres de hablantes que están en un excel.
#df = pandas.read_excel("/home/jmmiguez/proyectoAudio/audio/DiputadosDebatenElAcuerdoConElFMI3.xls")
#df = pandas.read_excel("/home/jmmiguez/proyectoAudio/audio/prueba_004.xls")

df = pandas.read_excel("/content/drive/MyDrive/proyectoAudio/audio/prueba_009.xls")

count = df.shape[0]


for label, name in enumerate(class_names):
#    print("Name={}- Label={}".format(name,label,))

    for index, row in df.iterrows():        
        #print("==>son iguales nombre={} y nombre={}".format(row[2],name))

        if (row[2]==name):
            print("Procesando al hablante {}".format(row[2],))
            dir_path = Path(DATASET_AUDIO_PATH) / row[2]    
            speaker_sample_paths = [
                os.path.join(dir_path, filepath)
                for filepath in os.listdir(dir_path)
                if filepath.endswith(".wav")
            ]
            audio_paths += speaker_sample_paths
            labels += [label] * len(speaker_sample_paths)

        if index == count - 1:
            break



#for label, name in enumerate(class_names):
#    print("Procesando al hablante {}".format(name,))
#    dir_path = Path(DATASET_AUDIO_PATH) / name
#    speaker_sample_paths = [
#        os.path.join(dir_path, filepath)
#        for filepath in os.listdir(dir_path)
#        if filepath.endswith(".wav")
#    ]
#    audio_paths += speaker_sample_paths
#    labels += [label] * len(speaker_sample_paths)

print(
    "Encontrados {} archivos, pertenecientes a {} clases. \nlabels={}".format(len(audio_paths), len(class_names), labels)
)


# Sobreescribo: Porcentaje de muestras que voy a usar para validación
# originalmente, usaba VALID_SPLIT = 0.1
#VALID_SPLIT = 0.5

# Mezclado
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# Dividir entre entrenamiento y validación
num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Usando {} archivos para entrenamiento.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Usando {} archivos para validación.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Crear 2 datasets, uno para entrenamiento y otro para validación
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)


# Agrego el ruido al conjunto de entrenamiento
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Transformo las señales de audio a la frecuencia de dominio, usando 'audio_to_fft'
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Nombres de nuestras Clases: ['JavierMilei', 'VozViva', 'MicaelaAmore', '.ipynb_checkpoints', 'DrDanielLopezRosetti', 'JoseLuisMartinOvejero', 'JuanManuelMiguez', 'MarceloPabloCassaretto', 'FatimaFlorez', 'WalbertoAllende', 'JoseLuisGioja', 'ClaudioPoggi', 'LauraRodriguezMachado', 'VictorHugoRomero', 'AlejandroCacace', 'MargaritaStolbizer', 'RicardoLopezMurphy', 'GracielaCamaño', 'CarlosMarioGutierrez', 'MyriamBregman', 'AgustinDomingo', 'JoseLuisEspert', 'RolandoFigueroa', 'SilvanaGinocchio', 'RominaDelPla', 'PaulaOmodeo', 'LuisDiGiacomo', 'CarlosZapata', 'SergioGuillermoCasas', 'AlvaroGonzalez', 'GermanaFigueroaCasas', 'AlejandroVilca', 'MartinTetaz', 'RosanaBertone', 'MaximilianoFerraro', 'EnriqueEstevez', 'MiguelNanni', 'MaraBrawer', 'GustavoSantos', 'FedericoAngelini', 'RicardoBuryaile', 'JuanManuelPedrini', 'FacundoManes', 'DanyaTavela', 'MarioLeito', 'SilviaLospennato', 'SusanaGracielaLandriscini', 'FranciscoMonti', 'MariaEugeniaVidal', 'DiegoSartori', 'OmarDeMarchi', 'BlancaOsu

In [9]:

def residual_block(x, filters, conv_num=3, activation="relu"):
    # Atajo
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model.summary()

# Compilar el modelo, usando la tasa de aprendizaje por defecto de Adam.
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Agregar devoluciones de llamada:
# 'EarlyStopping' para dejar de entrenar cuando el modelo ya no mejora.
# 'ModelCheckPoint' mantener siempre el modelo que tiene el mejor val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation (Activation)        (None, 8000, 16)     0           ['conv1d_1[0][0]']               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 8000, 16)     784         ['activation[0][0]']             
                                                                                              

In [10]:
#sobreescribo EPOCHS = 1
#EPOCHS = 5

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/50
3/3 [==============================] - 37s 3s/step - loss: 3.2875 - accuracy: 0.5202 - val_loss: 0.8270 - val_accuracy: 0.8587
Epoch 2/50
3/3 [==============================] - 6s 2s/step - loss: 0.7724 - accuracy: 0.8383 - val_loss: 0.7614 - val_accuracy: 0.9348
Epoch 3/50
3/3 [==============================] - 6s 1s/step - loss: 0.4951 - accuracy: 0.8787 - val_loss: 0.2756 - val_accuracy: 0.9348
Epoch 4/50
3/3 [==============================] - 6s 2s/step - loss: 0.4041 - accuracy: 0.8760 - val_loss: 0.2406 - val_accuracy: 0.9348
Epoch 5/50
3/3 [==============================] - 6s 2s/step - loss: 0.3974 - accuracy: 0.9003 - val_loss: 0.2714 - val_accuracy: 0.9457
Epoch 6/50
3/3 [==============================] - 6s 1s/step - loss: 0.2904 - accuracy: 0.9353 - val_loss: 0.3377 - val_accuracy: 0.8804
Epoch 7/50
3/3 [==============================] - 6s 2s/step - loss: 0.2581 - accuracy: 0.9245 - val_loss: 0.1728 - val_accuracy: 0.9348
Epoch 8/50
3/3 [========================

In [11]:
print(model.evaluate(valid_ds))

3/3 [==============================] - 1s 226ms/step - loss: 0.0893 - accuracy: 0.9674
[0.08929096162319183, 0.967391312122345]


In [12]:
#redefino BATCH_SIZE, por si la cantidad de muestras es menor a 128
BATCH_SIZE = 10

SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)
test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

for audios, labels in test_ds.take(1):
    # Obtener la señal FFT
    ffts = audio_to_fft(audios)
    # Predecir
    y_pred = model.predict(ffts)
    # Tomar muestras aleatorias
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    
    print("rnd {}".format(rnd))
    
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        # Para cada muestra, imprimir la etiqueta verdadera y predicha, así como ejecutar la voz con su ruido
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

rnd [6 0 9 2 2 9 1 9 3 6]
Speaker: ValKilmerAI	Predicted: ValKilmerAI


Speaker: DonaldTrump	Predicted: DonaldTrump


Speaker: ValKilmerAI	Predicted: ValKilmerAI


Speaker: DonaldTrump	Predicted: DonaldTrump


Speaker: DonaldTrump	Predicted: DonaldTrump


Speaker: ValKilmerAI	Predicted: ValKilmerAI


Speaker: DonaldTrump	Predicted: DonaldTrump


Speaker: ValKilmerAI	Predicted: ValKilmerAI


Speaker: DonaldTrump	Predicted: DonaldTrump


Speaker: ValKilmerAI	Predicted: ValKilmerAI


In [ ]:
#------------------------------------------------------------------------------------------------------------------------------

In [31]:
import os
from pathlib import Path
from IPython.display import Audio

BATCH_SIZE = 1
#SAMPLES_TO_DISPLAY = 1

valid_audio_paths_vivo = []
valid_audio_paths_vivo += ["{}".format(DATASET_ROOT + '/audio/VozViva/VozViva.wav')]
valid_labels += []
test_ds = paths_and_labels_to_dataset(valid_audio_paths_vivo, [6])
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)
test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))
for audios, labels in test_ds.take(1):
    # Obtener la señal FFT
    ffts = audio_to_fft(audios)
    # Predecir
    y_pred = model.predict(ffts)
    max_pred_value=0
    idx_max_pred_value=0
    name_max_pred_value=""
    pred_name = class_names[np.argmax(y_pred)]
    print(pred_name)
    #print("2)  -- {} {} ".format(pred_name, y_pred) )
    for index, value in enumerate(y_pred[0]):
        if (value > 0):
          print("y_pred({})<={} {}".format(index,value, class_names[index]) )
        if(value>max_pred_value):
            max_pred_value=value
            idx_max_pred_value=index
            name_max_pred_value=pred_name
    print("1) proba.cercanía=[{}] orden_categoría={} nombre_categoría={}".format(max_pred_value, idx_max_pred_value, name_max_pred_value) )
    y_pred = np.argmax(y_pred, axis=-1)
    print("2) predicción(Voz en Vivo) = {} ... y_pred={} index={}".format(class_names[y_pred[0]], y_pred, index))
    print("3) registro (Voz en Vivo) = {} - {}".format(valid_audio_paths_vivo[0], y_pred,  ) )

    display(Audio( valid_audio_paths_vivo[0], rate=SAMPLING_RATE ) )    

DonaldTrump
y_pred(0)<=4.860433307798463e-23 JavierMilei
y_pred(1)<=1.9809358517498964e-27 VozViva
y_pred(2)<=2.449716329121686e-26 MicaelaAmore
y_pred(3)<=1.413499150753684e-26 .ipynb_checkpoints
y_pred(4)<=1.3950032591248876e-25 DrDanielLopezRosetti
y_pred(5)<=1.0713150135814194e-22 JoseLuisMartinOvejero
y_pred(6)<=3.316275612409367e-14 JuanManuelMiguez
y_pred(7)<=1.035753170032508e-18 MarceloPabloCassaretto
y_pred(8)<=7.656378012103475e-29 FatimaFlorez
y_pred(9)<=2.333379875706645e-22 WalbertoAllende
y_pred(10)<=1.0426850930075122e-21 JoseLuisGioja
y_pred(11)<=6.137617589609641e-19 ClaudioPoggi
y_pred(12)<=8.143040359971051e-25 LauraRodriguezMachado
y_pred(13)<=3.2638861601572904e-24 VictorHugoRomero
y_pred(14)<=1.7740823838833053e-18 AlejandroCacace
y_pred(15)<=4.69886316251464e-18 MargaritaStolbizer
y_pred(16)<=2.4291821417081246e-23 RicardoLopezMurphy
y_pred(17)<=8.214067138687256e-28 GracielaCamaño
y_pred(18)<=9.00226355586107e-23 CarlosMarioGutierrez
y_pred(19)<=4.292958165459